In [22]:
import pandas as pd
import numpy as np
from dis import dis
import math
import geopandas as gpd
import pandas as pd
import numpy
from shapely import wkt
from shapely import wkb
from shapely.geometry import Point

modified = pd.read_csv('../data/modified_data.csv')
modified['geometry'] = [Point(xy) for xy in zip(modified.lat, modified.lon)]
#modified[]
#modified['geometry'] = modified['geometry'].replace(' POINT ', 'POINT')


grunnkrets_stripped = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
busstops = pd.read_csv('../data/busstops_norway.csv')
bs = busstops

df = modified


def dist(lat, lon, df): 
    lat1 = abs(df['lat'] - lat)
    lon1 = abs(df['lon'] - lon)
    eucdist = np.sqrt(lon1**2 + lat1**2)
    print(type(eucdist))
    return eucdist


modified['geometry'].head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


0    POINT (59.743104058362704 10.204927885979)
1     POINT (59.9137594158249 10.7340307646896)
2     POINT (60.1647505705996 10.2546558936519)
3     POINT (67.28366871498261 14.379796237766)
4     POINT (63.3580678289847 10.3748324689957)
Name: geometry, dtype: object

In [ ]:
import geopandas as gpd
import shapely.geometry
from shapely.geometry import Point
import pandas as pd
def dist_to_all_meter(lat, lon, data):
    gdb = gpd.GeoDataFrame(
        geometry=gpd.GeoSeries.from_wkt(
            data["geometry"],
            crs="epsg:4386",
        )
    )
    point = Point(lat, lon)
    gdb["distance"] = gdb.to_crs(gdb.estimate_utm_crs())["geometry"].distance(
        gpd.GeoSeries([point for _ in range(len(gdb))], crs=gdb.crs).to_crs(
            gdb.estimate_utm_crs()
        )
    ) 
    return gdb
dist_to_all_meter(50, 10, modified)



In [ ]:
bs.dtypes

In [ ]:

def store_dist_lvl(distance):
    distance = 10
    df['num_stores_within_distance_and_same_lvl1'] = 0
    df['num_stores_within_distance_and_same_lvl2'] = 0
    df['num_stores_within_distance'] = 0
    num = 0 

    for index in range(len(df)):
        lat = df._get_value(index,'lat')
        lon = df._get_value(index,'lon')
        lvl1 = df._get_value(index, 'lv1')
        lvl2 = df._get_value(index, 'lv2')
        dist_to_all = dist(lat, lon, df)

        count_distance_and_lvl1 = 0 
        count_distance_and_lvl2 = 0 
        count_distance = 0 #only for distance
        iter = 0
        
        for number in dist_to_all:
            if number < distance:
                count_distance += 1
                if lvl1 == df._get_value(iter, 'lv1'):
                    count_distance_and_lvl1 += 1
                    if lvl2 == df._get_value(iter, 'lv2'):
                        count_distance_and_lvl2 += 1
            iter += 1

        df._set_value(index, 'num_stores_within_distance_and_same_lvl1', count_distance_and_lvl1)
        df._set_value(index, 'num_stores_within_distance_and_same_lvl2', count_distance_and_lvl2)
        df._set_value(index, 'num_stores_within_distance', count_distance)

        num+=1 #for runtime tracking only
        print(str(num) + "/12860")


In [ ]:
def num_busstops_wihin_distance_x(x):
    #busstops['geometry'] = gpd.GeoSeries.from_wkt(busstops['geometry'])
    busstops['lon'] = busstops.geometry.apply(lambda x: x.x)
    busstops['lat'] = busstops.geometry.apply(lambda x: x.y)
    df['num_busstops_within_distance'] = 0

    for index in range(len(df)):
        one_to_all = dist(df._get_value(index, 'lat'), df._get_value(index, 'lon'), busstops)
        one_to_all = one_to_all.to_frame()
        one_to_all.rename( columns={0 :'a'}, inplace=True)
        count = (one_to_all['a'] > x).sum()
        df._set_value(index, 'num_busstops_within_distance', count)
    




In [ ]:
store_dist_lvl(0.1)

In [ ]:
num_busstops_wihin_distance_x(0.1)

In [ ]:
df.to_csv('test.csv')
df.head()